In [1]:
import requests
import pandas as pd
from pprint import pprint
from bs4 import BeautifulSoup
import numpy as np
import time
import random

In [2]:
def get_resource(url):
    headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    while True:
        try:
            time.sleep(random.uniform(1, 3))
            r = requests.get(url, headers=headers)
            return r
        except:
            print("requests.get failed, so try again!")
            #time.sleep(5)
            #r = requests.get(url, headers=headers, timeout=5)
            #return r
            continue
        break

In [3]:
def parse_html(r):
    if r.status_code == requests.codes.ok:
        r.encoding = 'utf8'
        soup = BeautifulSoup(r.text, 'lxml')
    else:
        print("Http請求錯誤: "+url)
        soup = None
    return soup

In [19]:
#first Step: 先拿到車子的網址存在websites
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36'}
websites = []
for i in range(1,132): #4712筆, 131pages
    url=f'https://www.isave.com.tw/cars.aspx?pageindex={i}&sort_expr=car_yy&sort_dir=desc' #依照年份新到舊排列
    time.sleep(8)
    r=requests.get(url,headers=headers)
    soup=BeautifulSoup(r.text,'lxml')
    car_info=soup.find_all(class_='listing') 
    for i in car_info:
        websites.append(i.a.get('href'))

KeyboardInterrupt: 

In [19]:
len(set(websites))

4715

In [4]:
#pd_websites = pd.DataFrame(websites, columns=['isave_columns'])
#pd_websites.to_csv('isave_websites0706.csv', encoding='utf_8_sig')
#已把網站網址存在isave_websites0706.csv
websites = pd.read_csv('isave_websites0706.csv', index_col=0)
websites['isave_columns'][:3]

0    car.aspx?cid=195648
1    car.aspx?cid=195646
2    car.aspx?cid=193888
Name: isave_columns, dtype: object

In [18]:
#Second Step: Get the detail of the cars
#df_detail_basic=pd.DataFrame(columns=['車商', '車款', '車型', '年份', '縣市', '顏色', '排檔', '排氣量', '里程數'])
df_detail_basic=pd.DataFrame(columns=['dealer_name', 'car_brand', 'car_model', 'car_year', 'car_address', 'car_color', 'car_gear', 'car_cylinderVolume', 'car_mileage'])
#df_equipements=pd.DataFrame(columns=['皮椅','晶片系統','恆溫空調','VCD','ABS','CD','電動窗','中控鎖','電動後視鏡','天窗','霧燈','DVD','倒車顯影','衛星導航','TCS防滑','防盜','安全氣囊','定速','一手車','電動座椅','核桃木飾板','4WD','倒車雷達','行車電腦','總代理','MP3','柴油','ＨＩＤ頭燈','鋁圈'])
df_equipements=pd.DataFrame(columns=['皮椅','晶片系統','auto_AC','IVI_sys1','ABS','IVI_sys2','電動窗','中控鎖','電動後視鏡','Sunroof','霧燈','IVI_sys3','倒車顯影','衛星導航','TCS防滑','防盜','SRS','定速','一手車','electric_seat','核桃木飾板','4WD','倒車雷達','行車電腦','總代理','MP3','柴油','ＨＩＤ','鋁圈'])
df_others=pd.DataFrame(columns=[])


title_all = []
price = []
detail_basic_all = []
equipements_all = []
address_all = []
photo_all = []
certificated_all = []
id_all = []
number = 0
car_others = pd.DataFrame(columns=['car_photo', 'car_price', 'car_title', 'dealer_address', 'id', 'verified'])

     
for j in websites['isave_columns'][3000:]: #4717筆, 132pages
    url=f'https://www.isave.com.tw/{j}' 
    #time.sleep(6)
    r=get_resource(url)
    soup=parse_html(r)
    if soup.find(class_='price') is not None:  

        #price
        price=soup.find(class_='price').text.strip()
        

        #title
        title = soup.find_all(class_="car-page-small-title")
        title_all = BeautifulSoup(str(title), 'html.parser').text
        

        #photo
        if soup.find(id="royalSlider-block", class_="royalSlider rsDefault image") is not None:
            car_photo = soup.find(id="royalSlider-block", class_="royalSlider rsDefault image")
            car_photo2 = BeautifulSoup(str(car_photo), 'html.parser').a
            photo_all='https://www.isave.com.tw'+car_photo2.get('href')
        else:
            photo_all=''

        #df_detail_basic車商名...
        detail_basic = soup.find_all(class_='main-car-details')
        #區塊轉型為HTML字串，再給 bs4 進行一次宣告
        detail_basic_1 = BeautifulSoup(str(detail_basic), 'html.parser')
        detail_basic_2 = detail_basic_1.find_all(class_='option-content')
        tmp = []
        for z in detail_basic_2:
            tmp.append(z.text.strip())
        detail_basic_all.append(tmp)


        #df_equipements
        equipements = soup.find_all(class_='car-equipment')
        equipements2 = BeautifulSoup(str(equipements), 'html.parser')
        equipements3 = equipements2.find_all('li')
        tmp = []
        for k in equipements3:
            if k.img.get('src') == 'images/icon/isave_yes_icon.png':
                tmp.append('1')
            else:
                tmp.append('0')
        equipements_all.append(tmp)

        #seller_address
        if soup.find_all(class_='store-detail') is not None:
            address = soup.find_all(class_='store-detail')
            address2 = BeautifulSoup(str(address), 'html.parser')
            for i in address2.find_all('span'):
                if '地　　　　　址' in i.text:
                    address_all=i.text.strip().replace('地　　　　　址：　','')
        else:
            address_all=''
        #id
        id=j.replace("car.aspx?cid=","")
        #certificated
        try:
            if soup.find_all(class_="car-tags") is not None:
                certificated = soup.find_all(class_="car-tags")
                certificated2 = BeautifulSoup(str(certificated), 'html.parser').div.text
                if '認證車' in certificated2:
                    certificated_all='1'
                else:
                    certificated_all='0'
            else:
                certificated_all='0'
        except:
            certificated_all='0'
            continue

        number+=1    
        print(j+"  success! :"+str(number))
        car_others = car_others.append({'car_price': price, 'car_photo': photo_all, 'car_title': title_all, 'dealer_address': address_all, 'verified': certificated_all, 'id': id_all}, ignore_index=True)
     
df1 = pd.DataFrame(detail_basic_all, columns=df_detail_basic.columns)
df2 = pd.DataFrame(equipements_all, columns=df_equipements.columns) 
df_detail_basic = df_detail_basic.append(df1, ignore_index=True) 
df_equipements = df_equipements.append(df2, ignore_index=True)
car_database = pd.concat([df_detail_basic, df_equipements,car_others],axis =1)

car.aspx?cid=195081  success! :1
car.aspx?cid=194813  success! :2
car.aspx?cid=194753  success! :3
car.aspx?cid=194712  success! :4
car.aspx?cid=194691  success! :5
car.aspx?cid=194572  success! :6
car.aspx?cid=194407  success! :7
car.aspx?cid=194301  success! :8
car.aspx?cid=194291  success! :9
car.aspx?cid=194069  success! :10
car.aspx?cid=194024  success! :11
car.aspx?cid=193971  success! :12
car.aspx?cid=193895  success! :13
car.aspx?cid=193781  success! :14
car.aspx?cid=193708  success! :15
car.aspx?cid=193647  success! :16
car.aspx?cid=193559  success! :17
car.aspx?cid=193531  success! :18
car.aspx?cid=193416  success! :19
car.aspx?cid=193367  success! :20
car.aspx?cid=193231  success! :21
car.aspx?cid=193159  success! :22
car.aspx?cid=193149  success! :23
car.aspx?cid=192836  success! :24
car.aspx?cid=192373  success! :25
car.aspx?cid=192221  success! :26
car.aspx?cid=191135  success! :27
car.aspx?cid=190950  success! :28
car.aspx?cid=190949  success! :29
car.aspx?cid=190834  su

car.aspx?cid=190668  success! :239
car.aspx?cid=190561  success! :240
car.aspx?cid=190397  success! :241
car.aspx?cid=190332  success! :242
car.aspx?cid=189920  success! :243
car.aspx?cid=189783  success! :244
car.aspx?cid=189746  success! :245
car.aspx?cid=189733  success! :246
car.aspx?cid=189434  success! :247
car.aspx?cid=189257  success! :248
car.aspx?cid=189196  success! :249
car.aspx?cid=188938  success! :250
car.aspx?cid=188805  success! :251
car.aspx?cid=188398  success! :252
car.aspx?cid=188289  success! :253
car.aspx?cid=188202  success! :254
car.aspx?cid=187993  success! :255
car.aspx?cid=187966  success! :256
car.aspx?cid=187797  success! :257
car.aspx?cid=187623  success! :258
car.aspx?cid=186947  success! :259
car.aspx?cid=186702  success! :260
car.aspx?cid=186407  success! :261
car.aspx?cid=186323  success! :262
car.aspx?cid=186271  success! :263
car.aspx?cid=185712  success! :264
car.aspx?cid=185479  success! :265
car.aspx?cid=185346  success! :266
car.aspx?cid=185014 

car.aspx?cid=191158  success! :474
car.aspx?cid=190823  success! :475
car.aspx?cid=190814  success! :476
car.aspx?cid=190750  success! :477
car.aspx?cid=190181  success! :478
car.aspx?cid=189712  success! :479
car.aspx?cid=189668  success! :480
car.aspx?cid=189612  success! :481
car.aspx?cid=189375  success! :482
car.aspx?cid=188945  success! :483
car.aspx?cid=188930  success! :484
car.aspx?cid=188487  success! :485
car.aspx?cid=188381  success! :486
car.aspx?cid=188298  success! :487
car.aspx?cid=188079  success! :488
car.aspx?cid=187490  success! :489
car.aspx?cid=187463  success! :490
car.aspx?cid=187284  success! :491
car.aspx?cid=186871  success! :492
car.aspx?cid=184660  success! :493
car.aspx?cid=184497  success! :494
car.aspx?cid=184492  success! :495
car.aspx?cid=184321  success! :496
car.aspx?cid=184269  success! :497
car.aspx?cid=184249  success! :498
car.aspx?cid=183976  success! :499
car.aspx?cid=183837  success! :500
car.aspx?cid=182824  success! :501
car.aspx?cid=182135 

car.aspx?cid=186760  success! :709
car.aspx?cid=177071  success! :710
car.aspx?cid=171348  success! :711
car.aspx?cid=195531  success! :712
car.aspx?cid=194929  success! :713
car.aspx?cid=192947  success! :714
car.aspx?cid=192946  success! :715
car.aspx?cid=191585  success! :716
car.aspx?cid=188895  success! :717
car.aspx?cid=181670  success! :718
car.aspx?cid=145653  success! :719
car.aspx?cid=195488  success! :720
car.aspx?cid=194294  success! :721
car.aspx?cid=194130  success! :722
car.aspx?cid=193415  success! :723
car.aspx?cid=193375  success! :724
car.aspx?cid=192419  success! :725
car.aspx?cid=192363  success! :726
car.aspx?cid=191898  success! :727
car.aspx?cid=191788  success! :728
car.aspx?cid=191785  success! :729
car.aspx?cid=191339  success! :730
car.aspx?cid=190687  success! :731
car.aspx?cid=190589  success! :732
car.aspx?cid=190434  success! :733
car.aspx?cid=190428  success! :734
car.aspx?cid=190356  success! :735
car.aspx?cid=189761  success! :736
car.aspx?cid=189360 

car.aspx?cid=192796  success! :944
car.aspx?cid=192745  success! :945
car.aspx?cid=192604  success! :946
car.aspx?cid=192349  success! :947
car.aspx?cid=191902  success! :948
car.aspx?cid=190975  success! :949
car.aspx?cid=189881  success! :950
car.aspx?cid=188745  success! :951
car.aspx?cid=188430  success! :952
car.aspx?cid=188407  success! :953
car.aspx?cid=187860  success! :954
car.aspx?cid=186677  success! :955
car.aspx?cid=185531  success! :956
car.aspx?cid=185109  success! :957
car.aspx?cid=185108  success! :958
car.aspx?cid=184330  success! :959
car.aspx?cid=183289  success! :960
car.aspx?cid=181622  success! :961
car.aspx?cid=181334  success! :962
car.aspx?cid=180810  success! :963
car.aspx?cid=180497  success! :964
car.aspx?cid=179518  success! :965
car.aspx?cid=179290  success! :966
car.aspx?cid=178097  success! :967
car.aspx?cid=177923  success! :968
car.aspx?cid=177672  success! :969
car.aspx?cid=176699  success! :970
car.aspx?cid=176394  success! :971
car.aspx?cid=194352 

car.aspx?cid=192636  success! :1174
car.aspx?cid=192633  success! :1175
car.aspx?cid=192387  success! :1176
car.aspx?cid=191528  success! :1177
car.aspx?cid=190859  success! :1178
car.aspx?cid=190777  success! :1179
car.aspx?cid=190701  success! :1180
car.aspx?cid=190699  success! :1181
car.aspx?cid=190535  success! :1182
car.aspx?cid=189991  success! :1183
car.aspx?cid=189627  success! :1184
car.aspx?cid=188380  success! :1185
car.aspx?cid=187639  success! :1186
car.aspx?cid=183870  success! :1187
car.aspx?cid=180511  success! :1188
car.aspx?cid=177893  success! :1189
car.aspx?cid=175750  success! :1190
car.aspx?cid=173940  success! :1191
car.aspx?cid=172417  success! :1192
car.aspx?cid=159431  success! :1193
car.aspx?cid=155045  success! :1194
car.aspx?cid=155043  success! :1195
car.aspx?cid=195886  success! :1196
car.aspx?cid=195615  success! :1197
car.aspx?cid=195538  success! :1198
car.aspx?cid=194998  success! :1199
car.aspx?cid=194666  success! :1200
car.aspx?cid=194610  success

car.aspx?cid=176283  success! :1402
car.aspx?cid=176904  success! :1403
car.aspx?cid=188963  success! :1404
car.aspx?cid=181348  success! :1405
car.aspx?cid=180711  success! :1406
car.aspx?cid=176901  success! :1407
car.aspx?cid=181223  success! :1408
car.aspx?cid=191287  success! :1409
car.aspx?cid=184901  success! :1410
car.aspx?cid=195828  success! :1411
car.aspx?cid=195746  success! :1412
car.aspx?cid=195511  success! :1413
car.aspx?cid=195468  success! :1414
car.aspx?cid=195291  success! :1415
car.aspx?cid=194995  success! :1416
car.aspx?cid=194794  success! :1417
car.aspx?cid=194580  success! :1418
car.aspx?cid=194236  success! :1419
car.aspx?cid=194229  success! :1420
car.aspx?cid=194141  success! :1421
car.aspx?cid=193303  success! :1422
car.aspx?cid=193224  success! :1423
car.aspx?cid=190910  success! :1424
car.aspx?cid=190852  success! :1425
car.aspx?cid=190703  success! :1426
car.aspx?cid=190700  success! :1427
car.aspx?cid=190316  success! :1428
car.aspx?cid=189829  success

car.aspx?cid=194152  success! :1630
car.aspx?cid=182989  success! :1631
car.aspx?cid=171043  success! :1632
car.aspx?cid=153028  success! :1633
car.aspx?cid=190140  success! :1634
car.aspx?cid=178361  success! :1635
car.aspx?cid=148761  success! :1636
car.aspx?cid=190348  success! :1637
car.aspx?cid=177576  success! :1638
car.aspx?cid=188319  success! :1639
car.aspx?cid=182514  success! :1640
car.aspx?cid=189266  success! :1641
car.aspx?cid=182637  success! :1642
car.aspx?cid=167162  success! :1643
car.aspx?cid=147207  success! :1644
car.aspx?cid=194348  success! :1645
car.aspx?cid=194154  success! :1646
car.aspx?cid=194147  success! :1647
car.aspx?cid=194145  success! :1648
car.aspx?cid=187227  success! :1649
car.aspx?cid=185553  success! :1650
car.aspx?cid=194157  success! :1651
car.aspx?cid=161065  success! :1652
car.aspx?cid=189488  success! :1653
car.aspx?cid=185705  success! :1654
car.aspx?cid=192120  success! :1655
car.aspx?cid=166305  success! :1656
car.aspx?cid=161813  success

In [26]:
# car_database['adress'] = address_all
# car_database['car_price'] = price
# car_database['car_photo'] = photo_all
# car_database['car_title'] = title_all
# car_database['verified'] = certificated_all
# car_database['id'] = id_all
print(car_database.head())
#刪除空值
car_database = car_database.dropna(how='all')
#car_database.to_csv('car_isave_part1_0712.csv', encoding='utf_8_sig')
#car_database.to_csv('car_isave_part2_0712.csv', encoding='utf_8_sig')
#car_database.to_csv('car_isave_part3_0712.csv', encoding='utf_8_sig')
#car_database.to_csv('car_isave_part4_0712.csv', encoding='utf_8_sig')
#car_database.to_csv('car_isave_part5_0712.csv', encoding='utf_8_sig')
#在這邊出錯

  dealer_name   car_brand     car_model car_year car_address car_color  \
0        大信汽車  MITSUBISHI        FORTIS    2012年         新北市       白色系   
1        萬通汽車      LUXGEN            S5    2012年         台中市       黑色系   
2        紘展汽車       HONDA        ACCORD    2012年         台北市       灰色系   
3        熊本車庫      TOYOTA  CAMRY HYBRID    2012年         台中市       黑色系   
4        永昌汽車       VOLVO          XC60    2012年         新北市       白色系   

  car_gear car_cylinderVolume car_mileage 皮椅  ... MP3 柴油 ＨＩＤ 鋁圈  \
0      手自排             1798CC     82599KM  1  ...   1  0   0  1   
1       自排             1800CC     80000KM  0  ...   0  0   0  0   
2      手自排             2354CC     70000KM  1  ...   0  0   0  1   
3      手自排             2500CC    148039KM  0  ...   0  0   0  0   
4      手自排             1998CC    160000KM  1  ...   1  0   1  1   

                                           car_photo car_price  \
0  https://www.isave.com.tw/public/photo/1316834/...   29.80 萬   
1  https://www.isave

In [27]:
p1 = pd.read_csv('car_isave_part1_0712.csv', encoding='utf_8_sig', index_col=0)
p2 = pd.read_csv('car_isave_part2_0712.csv', encoding='utf_8_sig', index_col=0)
p3 = pd.read_csv('car_isave_part3_0712.csv', encoding='utf_8_sig', index_col=0)
p4 = pd.read_csv('car_isave_part4_0712.csv', encoding='utf_8_sig', index_col=0)
p5 = pd.read_csv('car_isave_part5_0712.csv', encoding='utf_8_sig', index_col=0)

car_isave = pd.concat([p1, p2, p3, p4, p5],axis =0)
#刪除空值
car_isave = car_isave.dropna(how='all')

In [28]:
car_isave.shape

(4714, 44)

In [29]:
car_isave.to_csv('car_isave_all_0712.csv', encoding='utf_8_sig')

In [30]:
car_isave.head()

,dealer_name,car_brand,car_model,car_year,car_address,car_color,car_gear,car_cylinderVolume,car_mileage,皮椅,...,MP3,柴油,ＨＩＤ,鋁圈,car_photo,car_price,car_title,dealer_address,id,verified
0,萬通汽車,LUXGEN,URX,2021年,台中市,白色系,手自排,1798CC,1KM,0.0,...,0.0,0.0,0.0,0.0,https://www.isave.com.tw/public/photo/1323773/...,98 萬,[領牌新車 7人旗艦款 全景天窗 全車環景],台中市西屯區環中路一段2280號,[],1.0
1,萬通汽車,LUXGEN,URX,2021年,台中市,白色系,手自排,1798CC,1KM,0.0,...,0.0,0.0,0.0,0.0,https://www.isave.com.tw/public/photo/1323761/...,98 萬,[7人座旗艦款 新車 新車價112萬],台中市西屯區環中路一段2280號,[],1.0
2,辛巴克有限公司,FORD,KUGA,2021年,台中市,白色系,手自排,1999CC,4275KM,1.0,...,1.0,1.0,1.0,1.0,https://www.isave.com.tw/public/photo/1302609/...,109 萬,[榮獲2020年 下半年SAVE認證優良店],台中市烏日區新興路193號,[],1.0
3,廣一汽車商行,SUZUKI,JIMNY,2021年,嘉義縣,灰色系,自排,1462CC,48KM,0.0,...,0.0,0.0,0.0,1.0,https://www.isave.com.tw/public/photo/1279404/...,已收訂,[好車盡在廣一SAVE卓越店新車車道偏移],嘉義縣水上鄉中山路二段1234號,[],1.0
4,大信汽車,TOYOTA,COROLLA CROSS,2021年,新北市,白色系,手自排,1798CC,3KM,1.0,...,1.0,0.0,0.0,1.0,https://www.isave.com.tw/public/photo/1276916/...,72.80 萬,[全新未開 新車80萬 現折7萬 免綁全險],新北市泰山區新北大道五段618號,[],1.0


In [31]:
#pb:id is empty
car_isave.columns

Index(['dealer_name', 'car_brand', 'car_model', 'car_year', 'car_address',
       'car_color', 'car_gear', 'car_cylinderVolume', 'car_mileage', '皮椅',
       '晶片系統', 'auto_AC', 'IVI_sys1', 'ABS', 'IVI_sys2', '電動窗', '中控鎖', '電動後視鏡',
       'Sunroof', '霧燈', 'IVI_sys3', '倒車顯影', '衛星導航', 'TCS防滑', '防盜', 'SRS', '定速',
       '一手車', 'electric_seat', '核桃木飾板', '4WD', '倒車雷達', '行車電腦', '總代理', 'MP3',
       '柴油', 'ＨＩＤ', '鋁圈', 'car_photo', 'car_price', 'car_title',
       'dealer_address', 'id', 'verified'],
      dtype='object')

In [46]:
id_all = []
for i in car_isave['car_photo']:
    tmp = str(i).replace("https://www.isave.com.tw/public/photo/","")
    id_all.append(tmp.split('/')[0])
print(id_all[:3])
car_isave['id'] = id_all

['1323773', '1323761', '1302609']


In [47]:
car_isave.to_csv('car_isave_all_final_v2.csv', encoding='utf_8_sig')